In [291]:
from bigml.api import BigML
from pandas import DataFrame, to_numeric
import numpy
from pprint import pprint
from math import isnan

In [292]:
big_ml_project = 'project/5d94a428eba31d460c00023f'
api = BigML(project=big_ml_project)

In [293]:
model_id = 'ensemble/5db3a3a55299631c8200a5ed'

In [294]:
input = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

In [295]:
if 'MonthlyIncome' not in input:
    input['MonthlyIncome'] = None

Apply all feature mods.

In [296]:
df  = DataFrame(input, index=[0])

mode_debt_ratio=0.062488
df.loc[:,'MonthlyIncome'].fillna(0, inplace=True)
df.loc[:,'DebtRatio'].fillna(mode_debt_ratio, inplace=True)
df.loc[:,'MonthlyDebt']=df['DebtRatio']*df['MonthlyIncome']
df.loc[df['MonthlyIncome'] == 0,'MonthlyDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio']
df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=0
income_positive = df['MonthlyIncome'] > df['MonthlyDebt']
df.loc[income_positive,'BalancedIncome']= df.loc[income_positive,'MonthlyIncome'] - df.loc[income_positive,'MonthlyDebt']
df.loc[:,'NumberOfDependents'].fillna(0, inplace=True)
df.loc[:,'NumberOfDependents']=to_numeric(df['NumberOfDependents'],downcast='integer')
df.loc[:,'IncomePerHouseholdMember']= df['BalancedIncome'] / (df['NumberOfDependents']+1)
df.loc[:,'LateCategory']="L0"
df.loc[df['NumberOfTimes90DaysLate'] == 98,'LateCategory']="L98"
df.loc[df['NumberOfTimes90DaysLate'] == 96,'LateCategory']="L96"
df.loc[(df['NumberOfTimes90DaysLate'] == 98) | (df['NumberOfTimes90DaysLate'] == 96),
        ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']]=None
weight=[1,2,3]
df.loc[:,'LateScore']=weight[2]*df['NumberOfTimes90DaysLate']+weight[1]*df['NumberOfTime60-89DaysPastDueNotWorse']+weight[0]*df['NumberOfTime30-59DaysPastDueNotWorse']
df.drop('NumberOfTime30-59DaysPastDueNotWorse', axis=1, inplace=True)
df.drop('NumberOfTime60-89DaysPastDueNotWorse', axis=1, inplace=True)
df.drop('NumberOfTimes90DaysLate', axis=1, inplace=True)
df.drop('NumberOfDependents', axis=1, inplace=True)

df

,RevolvingUtilizationOfUnsecuredLines,DebtRatio,age,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,MonthlyIncome,MonthlyDebt,BalancedIncome,IncomePerHouseholdMember,LateCategory,LateScore
0,0.017036,0,42,6,1,0,0,NaN,NaN,L0,1.0


In [297]:
input2 = df.iloc[0].to_dict()
pprint(input)

{'DebtRatio': 0,
 'MonthlyIncome': None,
 'NumberOfDependents': 1,
 'NumberOfOpenCreditLinesAndLoans': 6,
 'NumberOfTime30-59DaysPastDueNotWorse': 1,
 'NumberOfTime60-89DaysPastDueNotWorse': 0,
 'NumberOfTimes90DaysLate': 0,
 'NumberRealEstateLoansOrLines': 1,
 'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
 'age': 42}


In [298]:
for key in input2:
    if isinstance(input2[key], numpy.int64):
        input2[key] = int(input2[key])
    if isinstance(input2[key], numpy.float64):
        input2[key] = float(input2[key])
    if isnan(input2[key]):
        input2[key]= 'NA'
    print(type(input2[key]))
pprint(input2)

<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'str'>
<class 'str'>


TypeError: must be real number, not str

In [ ]:
model = api.get_ensemble(model_id)

In [ ]:
prediction = api.create_prediction(model, input2)
prediction

In [ ]:
print(f"prediction: {prediction['object']['output']}")
print(f"prediction: {prediction['object']['confidence']}")

0.93267